# Azure OpenAI Service Completion 基礎
Azure OpenAI Service には、さまざまなタスクに使用できる入力候補（Completion）エンドポイントが用意されています。 このエンドポイントは、シンプルでありながら強力なテキストイン、テキストアウト インターフェイスをすべての Azure OpenAI モデルに提供します。 入力候補を実行するには、**プロンプト**としてテキストを入力します。モデルは入力候補を生成し、コンテキストやパターンとの一致を試みます。「デカルトが言ったように、我思う故に」というプロンプトを API に提供したとします。このプロンプトに基づいて、Azure OpenAI は高い確率で入力候補エンドポイント「我あり」を返します。

入力候補を試してみる最も良い方法は、[Azure OpenAI Studio](https://oai.azure.com/) のプレイグラウンドを使用することです。

## 事前準備

この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる[承認済み](https://aka.ms/oai/access) Azure サブスクリプション
- Azure OpenAI Service への GPT-3.5 Turbo モデルのデプロイメント。このデモでは、API バージョン `2023-12-01-preview` を使用しています。デプロイ名はモデルと同じ「`gpt-35-turbo-instruct`」を使用しています。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI GPT-3.5 Turbo Instruct モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## パッケージのインストール


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 必要なライブラリと環境変数のインポート


## Azure OpenAI の設定
接続情報はセキュリティ面から直接記述するよりも、環境変数や [dotenv](https://pypi.org/project/python-dotenv/) からロードする方法をおすすめします。

In [ ]:
import os

#from dotenv import load_dotenv
#load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

#これは、モデルをデプロイしたときにデプロイメントに選んだカスタム名に対応します。gpt-35-turbo-instructディプロイメントを使用してください。
AZURE_OPENAI_INSTRUCT_DEPLOYMENT = "gpt-35-turbo-instruct"

## Hello world!

### テキスト生成 - アイデア提案
モデルで実行できる最も強力で最も簡単なタスクの 1 つは、入力の新しいアイデアやバージョンを生成することです。ミステリー小説を書いていて、ストーリーのアイデアが必要だとします。モデルにアイデアのリストを提供すると、モデルはそのリストにアイデアを追加しようとします。モデルは、ほんの少数の例に基づいて、ビジネス プラン、キャラクターの説明、マーケティング スローガンなどを作成することができます。

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions


In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
# Send a completion call to generate an answer
prompt = "アイスクリームショップのキャッチコピーを考えてください"
response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT, prompt=prompt, max_tokens=100
)
print(prompt)
print(response.choices[0].text)

上の例ではコンテキストが少ししか提供されないため、モデルが期待される結果を常に返すとは限らないのは正常です。 応答が予期しないまたは切り捨てられたように見える場合は、トークンの最大数 `max_tokens` を調整できます。


## コード補完
補完機能を使用して React コード コンポーネントの作成を支援する方法を示しています。まず、モデルにコードを送信します。開き括弧 `(` でコードの入力を停止します。モデルは不完全なコードを HeaderComponent の定数定義を補完するためのトリガーと解釈します。さらにモデルは対応する React ライブラリを理解しているため、このコード定義を補完できます。

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions#complete-partial-text-and-code-inputs

In [ ]:
# Generation prompt
prompt = """
import React from 'react';
const HeaderComponent = () => (
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## テキストの要約
モデルはテキストのコンテキストを把握し、それをさまざまな方法で言い換えることができます。このデモでは、モデルがテキストのブロックを受け取り、小学生でも理解できる説明を作成します。この例は、モデルが言語を深く理解していることを示しています。

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions#summarize-text

In [ ]:
# Generation prompt
prompt = """
以下の説明を 100 字程度で要約してください。

###
中性子星は質量が太陽程度、直径20 km程度、大気の厚さはわずか1 m程度で、中性子が主な成分の天体である。
密度は太陽の10^14倍以上もあるとされている。およそ10^9 t/cm3とその桁外れに大きい密度のため、中性子星の
表面重力は地球の表面重力の2×10^11倍もの大きさがあり、脱出速度は 1/3 c にも達する。中性子星は大質量の
恒星の超新星爆発によってその中心核が圧縮された結果形成されるが、中性子星として存在できる質量にはトルマン
・オッペンハイマー・ヴォルコフ限界と呼ばれる上限値があり、それを超えるとブラックホールとなる。
上限の質量は、理論的に太陽質量の1.5倍から2.5倍の範囲にあると考えられており、2010年に約1.97倍の中性子星、
2013年には約2.01倍の中性子星が確認されている。下限は太陽質量の0.1倍から0.2倍程度。
###

"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

In [ ]:
# Generation prompt
prompt = """
以下の説明を6歳の子供でも理解できるように分かりやすく要約してください。
子供が分からない単語は、分かりやすい言葉に置き換えてください。

###
中性子星は質量が太陽程度、直径20 km程度、大気の厚さはわずか1 m程度で、中性子が主な成分の天体である。
密度は太陽の10^14倍以上もあるとされている。およそ10^9 t/cm3とその桁外れに大きい密度のため、中性子星の
表面重力は地球の表面重力の2×10^11倍もの大きさがあり、脱出速度は 1/3 c にも達する。中性子星は大質量の
恒星の超新星爆発によってその中心核が圧縮された結果形成されるが、中性子星として存在できる質量にはトルマン
・オッペンハイマー・ヴォルコフ限界と呼ばれる上限値があり、それを超えるとブラックホールとなる。
上限の質量は、理論的に太陽質量の1.5倍から2.5倍の範囲にあると考えられており、2010年に約1.97倍の中性子星、
2013年には約2.01倍の中性子星が確認されている。下限は太陽質量の0.1倍から0.2倍程度。
###

"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

Azure OpenAI API は対話のたびに新しい出力を生成するため、表示される入力候補の結果は異なる場合があります。 プロンプトが同じであっても、API を呼び出すたびに少しずつ異なる入力候補が得られる可能性があります。 この動作は、`temperature` の設定で制御できます。

## テキストの分類
テキスト分類器を作成するには、タスクの説明を提示し、いくつかの例を示します。この例では、テキストメッセージのセンチメントを分類する方法をモデルに示しています。センチメントとは、テキスト内の全体的な感情や表現のことです。

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions

In [ ]:
prompt = """
これはテキストメッセージのセンチメント分類器です。

Message: "新しいアドベンチャー映画が大好きです！"
Sentiment: ポジティブ

Message: "携帯のバッテリーが切れるのは嫌だ" 
Sentiment: 否定的

Message: "今日は👍"
Sentiment: ポジティブ

Message: "これは記事へのリンクです"
Sentiment: 中立

Message: "この新しいミュージックビデオは非現実的だ"
Sentiment: 
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

### 1 回の API 呼び出しから複数の結果を得る
分類器の作成方法を理解したので、最初のデモを拡張して、効率を高めましょう。分類器を使用して、1 回の API 呼び出しから複数の結果を取得できます。

In [ ]:
prompt = """
これはテキストメッセージのセンチメント分類器です。

Message: "新しいアドベンチャー映画が大好きです！"
Sentiment: ポジティブ

Message: "携帯のバッテリーが切れるのは嫌だ" 
Sentiment: 否定的

Message: "今日は👍"
Sentiment: ポジティブ

Message: "これは記事へのリンクです"
Sentiment: 中立

メッセージテキスト
1. "新しいアドベンチャー映画が大好きだった！"
2. "携帯の電池が切れるのが嫌だ"
3. "今日は👍"
4. "これが記事へのリンクです"
5. "この新しいミュージックビデオは非現実的だ"

メッセージのセンチメント評価
1: ポジティブ
2: 否定的
3: ポジティブ
4: 中立
5: ポジティブ

メッセージテキスト
1. "彼は宿題が嫌いです"
2. 「タクシーが遅れている。彼女は怒っている😠"
3. 「週末が待ち遠しい！！」
4. "うちの猫がかわいい ❤️❤️"
5. "チョコバナナ食べてみよう"

メッセージのセンチメント評価
1:
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## テキストの翻訳
日本語のフレーズをフランス語に変換する方法をモデルに指示します。

In [ ]:
prompt = """
Japanese: 私はフランス語は話せません。
French: Je ne parle pas français.
Japanese: また会おう！
French: À tout à l'heure!
Japanese: いいレストランはどこですか？
French: Où est un bon restaurant?
Japanese: 空いている部屋はありますか？
French: Quelles chambres avez-vous de disponible?
Japanese:最寄りの駅はどこですか？
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

### 絵文字への変換
映画の題名をテキストから絵文字に変換します。この例は、モデルがパターンを把握して、他の文字を使用するだけの適応力を持っていることを示しています。

In [ ]:
prompt = """
Carpool Time: 👨👴👩🚗🕒
Robots in Cars: 🚗🤖
Super Femme: 👸🏻👸🏼👸🏽👸🏾👸🏿
Webs of the Spider: 🕸🕷🕸🕸🕷🕸
The Three Bears: 🐻🐼🐻
Mobster Family: 👨👩👧🕵🏻‍♂️👲💥
Arrows and Swords: 🏹🗡🗡🏹
Snowmobiles:
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## 質問応答（Q&A）
コンテキストを使って業界独自の文書、企業内 FAQ など、GPT モデルが知らない様々な文書も質問応答の対象にできます。

In [ ]:
prompt = """
以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。

コンテキスト:
###
Surface Book が空の状態から完全に充電されるまで、2 ～ 4 時間かかります。
Surface Book を充電しながらゲームやビデオ ストリーミングのような電力消費の
多い活動に Surface を使用している場合、さらに時間がかかる可能性があります。
電源アダプターに付いている USB ポートを使って、Surface Book の充電中に
スマートフォンなどの他のデバイスを充電することもできます。電源アダプターの
USB ポートは充電専用であり、データ転送用ではありません。
###

質問: Surface Book の充電時間を節約するにはどうすればよいか。
回答: 
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.0,
    top_p=0.9,
)
print(response.choices[0].text)

### テンプレート化
プロンプトをテンプレート化することで、プログラムとの統合や管理をしやすくすることができます。

In [ ]:
prompt_template = """
以下のテキストを使って下記の質問に答えてください。もし答えがない場合には、「私は知らない」と答えてください。

コンテキスト: 
###
{input}
###

質問: {question}
回答: 
"""

prompt = prompt_template.format(
    input="Surface Book が空の状態から完全に充電されるまで、2 ～ 4 時間かかります。Surface Book を充電しながらゲームやビデオ ストリーミングのような電力消費の多い活動に Surface を使用している場合、さらに時間がかかる可能性があります。電源アダプターに付いている USB ポートを使って、Surface Book の充電中にスマートフォンなどの他のデバイスを充電することもできます。電源アダプターの USB ポートは充電専用であり、データ転送用ではありません。",
    question="Surface Book の充電時間を節約するにはどうすればよいか。",
)
print(prompt)

In [ ]:
response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.0,
    top_p=0.9,
)
print(response.choices[0].text)

## エンティティ抽出
テキストの中からエンティティ（名前や地名、組織など）を抽出するデモです。

In [ ]:
prompt = """
下記のEメールの内容から、差出人の名前とその人の住所を抽出してください。

###
鈴木さん、セミナーで一緒にお話できて本当嬉しいです。加藤さんの話も、とてもすばらしかったです。
書籍の件、ありがとうございます。私の住所は、東京都港区港南２丁目１６−３です。

山田太郎
###
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)

## Zero-shot CoT（思考の連鎖）
例示なしでタスクの解法を誘導する手法（「一歩ずつ順番に考えてください」原文「Let's think step by step」によって正しい回答へ誘導）です。ユーザーの複雑な質問や指示をタスクに分解して、それぞれのタスクごとに解決策を講じることができる非常に重要なテクニックです。


In [ ]:
prompt = """
質問 :ジョンはりんごを5個持っていて、2個食べて、さらに5個買って、3個を友達に上げたとしたら、ジョンはりんごを何個持っていますか？
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)

In [ ]:
prompt = """
質問 :ジョンはりんごを5個持っていて、2個食べて、さらに5個買って、3個を友達に上げたとしたら、ジョンはりんごを何個持っていますか？

一歩ずつ順番に考えてください。

"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)